# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# Write your code below.
%load_ext dotenv
%dotenv 


In [2]:
import dask.dataframe as dd

c:\Users\markn\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\_pyarrow_compat.py:15: FutureWarning: Minimal version of pyarrow will soon be increased to 14.0.1. You are using 11.0.0. Please consider upgrading.
  warnings.warn(


+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.

# Get the PRICE_DATA path from environment
price_data_path = os.getenv("PRICE_DATA")

# Use glob to find all .parquet files recursively
parquet_files = glob(os.path.join(price_data_path, "**", "*.parquet"), recursive=True)

# check how many were found
print(f"Found {len(parquet_files)} parquet files.")


Found 2045 parquet files.


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Close:
    
    - `returns`: (Close / Close_lag_1) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [4]:
# Write your code below.

dd_prices = dd.read_parquet(parquet_files)


# Step 1: Define the transformation function
def create_features(df):
    df = df.sort_values("Date")
    df["Close_lag_1"] = df["Close"].shift(1)
    df["Adj_Close_lag_1"] = df["Adj_Close"].shift(1)
    df["returns"] = (df["Close"] / df["Close_lag_1"]) - 1
    df["hi_lo_range"] = df["High"] - df["Low"]
    return df

# Step 2: Apply the function to each ticker group
dd_feat = dd_prices.groupby("ticker").apply(create_features, meta = {
    'ticker': 'object',
    'Date': 'datetime64[ns]',
    'Open': 'float64',
    'High': 'float64',
    'Low': 'float64',
    'Close': 'float64',
    'Adj_Close': 'float64',
    'Volume': 'float64',
    'Year': 'int64',
    'source': 'object',
    'Close_lag_1': 'float64',
    'Adj_Close_lag_1': 'float64',
    'returns': 'float64',
    'hi_lo_range': 'float64'
})

+ Convert the Dask data frame to a pandas data frame. 
+ Add a new feature containing the moving average of `returns` using a window of 10 days. There are several ways to solve this task, a simple one uses `.rolling(10).mean()`.

(3 pt)

In [ ]:
# Write your code below.

# Convert Dask DataFrame to pandas DataFrame

df_feat = dd_feat.compute()

# Sort by ticker and Date for correct rolling calculation
df_feat = df_feat.sort_values(["ticker"])

# Add 10-day moving average of returns per ticker
df_feat["returns_ma_10"] = (
    df_feat.groupby("ticker")["returns"]
    .transform(lambda x: x.rolling(10).mean())
)

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)

Was Pandas necessary for moving average return?
Not strictly necessary; Pandas is convenient for moderate-sized data with its simple rolling().mean(). Alternatives like NumPy or raw Python work but are less intuitive. Conversion to Pandas is justified if you’re already using its ecosystem or need its time-series features.

Would Dask be better? Why?
Dask is better for large datasets (>1GB) or distributed computing, as it handles out-of-memory data and parallelizes tasks. For smaller datasets or single-machine workflows, Pandas is faster and simpler. Use Dask only if memory or scaling needs arise.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.